#Import Libraries

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
!pip install transformers -q
!pip install nltk
!pip install pandarallel 

In [20]:
! pip install annoy -q

In [21]:
# Importing Libraries
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np
import string
import re
from tqdm.auto import tqdm
tqdm.pandas()
import math
import _pickle as cPickle
import nltk
nltk.download('punkt')

# Checking to see if CUDA is available if so storing details in device

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
import psutil

workers = psutil.cpu_count()
# psutil.cpu_count(logical=False)

In [23]:
import pandas as pd

from datetime import datetime
from collections import Counter
import torch
from transformers import AutoTokenizer, AutoModel, BertModel
from sklearn.metrics.pairwise import cosine_similarity


import re  
from gensim.models import Word2Vec    #Import gensim Word2Fec
from sklearn.decomposition import PCA #Grab PCA functions
import numpy as np

#Plot helpers
import matplotlib.pyplot as plt

#Enable matplotlib to be interactive (zoom etc)
import ast

from pandarallel import pandarallel

from pandarallel.utils import progress_bars
progress_bars.is_notebook_lab = lambda : True

pandarallel.initialize(progress_bar=True, nb_workers=workers, use_memory_fs=False)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


#Loading Data and Lexicon

In [24]:
#new ones_collected by passing through the emobert
# df = pd.read_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/NRC Emotion Intensity.csv")
# df = pd.read_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/NRC-Emotion-Intensity-Lexicon-v2.csv")
# df = pd.read_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/NRC-Emotion-Intensity-Lexicon-lda.csv")
df = pd.read_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/NRC-Emotion-Intensity-Lexicon-without_trust.csv")

# df['embedding'] = df['embedding'].tolist()
df['embedding'] = [ast.literal_eval(i) for i in df['embedding'].values.tolist()]
print(df.head())
print(df.columns)
df = df.dropna()

         word  ...                                          embedding
0    aaaaaaah  ...  [0.3540674149990082, 0.7942342162132263, 0.423...
1   abandoned  ...  [-0.5577418804168701, 0.015890369191765785, -0...
2  aberration  ...  [-0.5287905931472778, -0.1950991153717041, -0....
3       abhor  ...  [-0.6878913640975952, 0.1946220099925995, -1.3...
4   abhorrent  ...  [-0.3238641023635864, 0.3315010368824005, -1.2...

[5 rows x 4 columns]
Index(['word', 'emotion', 'emotion-intensity-score', 'embedding'], dtype='object')


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2273 entries, 0 to 2272
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   word                     2273 non-null   object 
 1   emotion                  2273 non-null   object 
 2   emotion-intensity-score  2273 non-null   float64
 3   embedding                2273 non-null   object 
dtypes: float64(1), object(3)
memory usage: 88.8+ KB


In [26]:
len(df['embedding'][0])

768

In [27]:
from annoy import AnnoyIndex
import random

f = 768
t = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed

# super fast, will just mmap the file
# t.load(r'/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/angular_nrc_embedding_space_V6.ann') 
# t.load(r'/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/angular_nrc_embedding_space_words_lda.ann') 
t.load(r'/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/angular_nrc_embedding_space_words_without_trust.ann') 

True

In [28]:
t.get_n_items()

2273

#Liar

In [ ]:
dff = pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/LIAR/Liar_all.csv")
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12791 entries, 0 to 12790
Data columns (total 15 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  12791 non-null  object 
 1   label                               12791 non-null  object 
 2   statement                           12791 non-null  object 
 3   subject(s)                          12789 non-null  object 
 4   speaker                             12789 non-null  object 
 5   speaker_job_title                   9224 non-null   object 
 6   state                               10042 non-null  object 
 7   party                               12789 non-null  object 
 8   credit_history_count_barely_true    12789 non-null  float64
 9   credit_history_count_false          12789 non-null  float64
 10  credit_history_count_half_true      12789 non-null  float64
 11  credit_history_count_mostly_true    12789

In [ ]:
dff.head()

,ID,label,statement,subject(s),speaker,speaker_job_title,state,party,credit_history_count_barely_true,credit_history_count_false,credit_history_count_half_true,credit_history_count_mostly_true,credit_history_count_pants_on_fire,context,split
0,11972.json,true,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30.0,30.0,42.0,23.0,18.0,Radio interview,test
1,11685.json,false,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2.0,1.0,0.0,0.0,0.0,a news conference,test
2,11096.json,false,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63.0,114.0,51.0,37.0,61.0,comments on ABC's This Week.,test
3,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1.0,1.0,3.0,1.0,1.0,a radio show,test
4,9524.json,pants-fire,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5.0,7.0,2.0,2.0,7.0,a web video,test


In [ ]:
dff = dff[['label','statement','ID','split']]

In [ ]:
dff.dropna(subset = ['statement'], how='all', inplace=True)
dff.reset_index(drop=True,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
dff['statement'][0]

'Building a wall on the U.S.-Mexico border will take literally years.'

#Politifact

In [ ]:
col_names = ['cred_label', 'claim_id', 'claim_text', 'claim_source', 'evidence', 'evidence_source' ]

In [ ]:
dff = pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/Politifact/Politifact.tsv", sep='\t', names = col_names)
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29556 entries, 0 to 29555
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cred_label       29556 non-null  object
 1   claim_id         29556 non-null  object
 2   claim_text       29556 non-null  object
 3   claim_source     29556 non-null  object
 4   evidence         29556 non-null  object
 5   evidence_source  29556 non-null  object
dtypes: object(6)
memory usage: 1.4+ MB


In [ ]:
dff['cred_label'].unique()

array(['True', 'Half-True', 'Mostly True', 'False', 'Pants on Fire!',
       'Mostly False'], dtype=object)

In [ ]:
dff.head()

,cred_label,claim_id,claim_text,claim_source,evidence,evidence_source
0,True,2012_oct_08_barack-obama_obama-says-tax-code-r...,federal tax code loopholes giving incentives c...,barack obama,for firms moving overseas in order to create a...,foxnews.com
1,True,2012_oct_08_barack-obama_obama-says-tax-code-r...,federal tax code loopholes giving incentives c...,barack obama,get a tax break specifically by outsourcing jo...,newslines.org
2,True,2012_oct_08_barack-obama_obama-says-tax-code-r...,federal tax code loopholes giving incentives c...,barack obama,confusing clashes over taxes in wednesday s pr...,wsj.com
3,True,2012_oct_08_barack-obama_obama-says-tax-code-r...,federal tax code loopholes giving incentives c...,barack obama,support on this bill in a time of tight budget...,senate.gov
4,True,2012_oct_08_barack-obama_obama-says-tax-code-r...,federal tax code loopholes giving incentives c...,barack obama,tax a lower rate for american manufacturing an...,archives.gov


In [ ]:
dff = dff.drop_duplicates(subset='claim_text', keep="first")
dff.reset_index(drop=True,inplace=True)
dff = dff[['cred_label','claim_id','claim_text']]
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3529 entries, 0 to 3528
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   cred_label  3529 non-null   object
 1   claim_id    3529 non-null   object
 2   claim_text  3529 non-null   object
dtypes: object(3)
memory usage: 82.8+ KB


In [ ]:
dff.dropna(subset = ['claim_text'], how='all', inplace=True)
dff.reset_index(drop=True,inplace=True)

In [ ]:
dff['claim_text'][0]

'federal tax code loopholes giving incentives companies shipping jobs overseas'

#FakeNewsNet

In [29]:
dff = pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/FakeNewsNet/FakeNewsNet_All.csv")
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23196 entries, 0 to 23195
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         23196 non-null  object
 1   news_url   22866 non-null  object
 2   title      23196 non-null  object
 3   tweet_ids  21695 non-null  object
 4   label      23196 non-null  object
 5   source     22140 non-null  object
 6   id_1       23196 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 1.2+ MB


In [30]:
dff['source'].fillna('politifact',inplace=True)

In [31]:
dff.head()

,id,news_url,title,tweet_ids,label,source,id_1
0,gossipcop-2493749932,www.dailymail.co.uk/tvshowbiz/article-5874213/...,Did Miley Cyrus and Liam Hemsworth secretly ge...,284329075902926848\t284332744559968256\t284335...,Fake,gossipcop,0
1,gossipcop-4580247171,hollywoodlife.com/2018/05/05/paris-jackson-car...,Paris Jackson & Cara Delevingne Enjoy Night Ou...,992895508267130880\t992897935418503169\t992899...,Fake,gossipcop,1
2,gossipcop-941805037,variety.com/2017/biz/news/tax-march-donald-tru...,Celebrities Join Tax March in Protest of Donal...,853359353532829696\t853359576543920128\t853359...,Fake,gossipcop,2
3,gossipcop-2547891536,www.dailymail.co.uk/femail/article-3499192/Do-...,Cindy Crawford's daughter Kaia Gerber wears a ...,988821905196158981\t988824206556172288\t988825...,Fake,gossipcop,3
4,gossipcop-5476631226,variety.com/2018/film/news/list-2018-oscar-nom...,Full List of 2018 Oscar Nominations – Variety,955792793632432131\t955795063925301249\t955798...,Fake,gossipcop,4


In [32]:
dff = dff[['title','label','id_1','source','id']]

In [33]:
dff['title'][0]

'Did Miley Cyrus and Liam Hemsworth secretly get married?'

In [34]:
dff.dropna(subset = ['title'], how='all', inplace=True)
dff.reset_index(drop=True,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#Covid CodeLab

In [ ]:
dff = pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/CodaLab Covid/Constraint_English_All.csv")
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10700 entries, 0 to 10699
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      10700 non-null  object
 1   tweet   10700 non-null  object
 2   label   10700 non-null  object
 3   split   10700 non-null  object
dtypes: object(4)
memory usage: 334.5+ KB


In [ ]:
dff.head()

,id,tweet,label,split
0,test1,Our daily update is published. States reported...,real,test
1,test2,Alfalfa is the only cure for COVID-19.,fake,test
2,test3,President Trump Asked What He Would Do If He W...,fake,test
3,test4,States reported 630 deaths. We are still seein...,real,test
4,test5,This is the sixth time a global health emergen...,real,test


In [ ]:
dff['tweet'][0]

'Our daily update is published. States reported 734k tests 39k new cases and 532 deaths. Current hospitalizations fell below 30k for the first time since June 22. https://t.co/wzSYMe0Sht'

In [ ]:
dff.dropna(subset = ['tweet'], how='all', inplace=True)
dff.reset_index(drop=True,inplace=True)

#Data Cleaning

In [35]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23196 entries, 0 to 23195
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   23196 non-null  object
 1   label   23196 non-null  object
 2   id_1    23196 non-null  int64 
 3   source  23196 non-null  object
 4   id      23196 non-null  object
dtypes: int64(1), object(4)
memory usage: 906.2+ KB


In [36]:
def lower(text):
  return text.lower()

dff['title'] = dff['title'].apply(lower)

In [37]:
# Remove Duplicates

duplicate_count = len(dff['title'])-len(dff['title'].drop_duplicates())

print('Duplicate count:', duplicate_count)
print('Total records before remove duplicates:', dff.shape[0])


dff = dff.drop_duplicates(subset='title', keep="first")

print('updated record count:', dff.shape[0])

Duplicate count: 1478
Total records before remove duplicates: 23196
updated record count: 21718


In [38]:
import string
import re
import nltk.data

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_nonascii(sent):
  return "".join([i for i in sent if i.isascii()])

def remove_punctuations(text):
  res = re.sub(r'[^\w\s]', ' ', text)
  return res

def remove_emoji(string):
    '''
    Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
    remove emojis
    '''
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def strip_all_tags(text):
    '''
    uses this to remove any mentions and hash tags
    '''
    entity_prefixes = ['@','#']
    words = []
    for word in text.split():
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

def remove_num(text):
  return "".join([c for c in text if not c.isdigit()])

def remove_mul_space(text):
  return " ".join(text.split())

In [39]:
def clean(tokens):
  refined_tokens = []
  tokens = remove_urls(tokens)
  tokens = remove_emoji(tokens)
  tokens = strip_all_tags(tokens)
  tokens = remove_nonascii(tokens)
  tokens = remove_punctuations(tokens)
  tokens = remove_num(tokens)
  tokens = remove_mul_space(tokens)
  return tokens


dff['tokens'] = dff['title'].apply(clean)
dff = dff.reset_index(drop=True)
dff['index'] = dff.index

In [40]:
dff.tail()

,title,label,id_1,source,id,tokens,index
21713,the obameter: introduce a comprehensive immigr...,Real,23189,politifact,politifact6931,the obameter introduce a comprehensive immigra...,21713
21714,"trump asking congress, not mexico, to pay for ...",Real,23190,politifact,politifact13619,trump asking congress not mexico to pay for bo...,21714
21715,flake: “religious tests should have no place i...,Real,23191,politifact,politifact14731,flake religious tests should have no place in ...,21715
21716,deputy director of national health statistics ...,Real,23193,politifact,politifact1576,deputy director of national health statistics ...,21716
21717,romneys prolife conversion myth or reality jun...,Real,23194,politifact,politifact4720,romneys prolife conversion myth or reality june,21717


In [41]:
# # Next we remove stop words and short words.
# # For stop words we use the SMART stop word list here
# # Initially we import the text file and create the list.

# # stop = []
# # with open("/content/drive/Shareddrives/FYP - knk/Resources/SMART_STOP_WORDS.txt", "r") as f:
# #   for word in f:
# #       # Here we remove the apostrophe as well
# #       stop.append(word.strip().replace("'",""))
# # stop.extend(['nan', 'coronavirus','covid','covid19', 'vaccine', 'false', 'selfcovidvaccine', 'vaccines', 'vaccinevaccine', 'corona', 'pandemic'])

# dff['refined_tokens'] = dff['tokens'].apply(remove_stop_words_and_short_words)


In [42]:
def extract_trigrams(sentence):
  window_size = 3
  sentence_pieces = []
  sentence_tokens = sentence.split(' ')
  i = 0
  while(i<len(sentence_tokens)):
    sliding_piece = ' '.join(sentence_tokens[i:i+window_size])
    i = i + window_size
    sentence_pieces.append(sliding_piece)
  return sentence_pieces

dff['refined_tokens'] = dff['tokens'].apply(extract_trigrams)

In [43]:
dff.tail()

,title,label,id_1,source,id,tokens,index,refined_tokens
21713,the obameter: introduce a comprehensive immigr...,Real,23189,politifact,politifact6931,the obameter introduce a comprehensive immigra...,21713,"[the obameter introduce, a comprehensive immig..."
21714,"trump asking congress, not mexico, to pay for ...",Real,23190,politifact,politifact13619,trump asking congress not mexico to pay for bo...,21714,"[trump asking congress, not mexico to, pay for..."
21715,flake: “religious tests should have no place i...,Real,23191,politifact,politifact14731,flake religious tests should have no place in ...,21715,"[flake religious tests, should have no, place ..."
21716,deputy director of national health statistics ...,Real,23193,politifact,politifact1576,deputy director of national health statistics ...,21716,"[deputy director of, national health statistic..."
21717,romneys prolife conversion myth or reality jun...,Real,23194,politifact,politifact4720,romneys prolife conversion myth or reality june,21717,"[romneys prolife conversion, myth or reality, ..."


In [44]:
total_tokens = 0
total_length = len(dff)
for index, row in dff.iterrows():
  total_tokens+=len(row['refined_tokens'])

average_length = total_tokens/total_length
print(average_length)

4.098996224330048


#Mean Pooling

In [45]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    # print('ime',input_mask_expanded)
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    # print('se',sum_embeddings)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [46]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [47]:
tokenizer = AutoTokenizer.from_pretrained("/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/go_model")
model = AutoModel.from_pretrained("/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/go_model")

Some weights of the model checkpoint at /content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/go_model were not used when initializing DistilBertModel: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [48]:
model.to(device)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [49]:
def get_mean_pooling_emb_tokenized(sentences):
  encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')
  encoded_input.to(device)
  #Compute token embeddings
  with torch.no_grad():
      model_output = model(**encoded_input)

  sentence_embeddings_raw = mean_pooling(model_output, encoded_input['attention_mask'])
  sentence_embeddings = sentence_embeddings_raw.tolist()

  return sentence_embeddings 

In [50]:
def get_mean_pooling_emb(sentences):
  encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')
  encoded_input.to(device)
  #Compute token embeddings
  with torch.no_grad():
      model_output = model(**encoded_input)

  sentence_embeddings_raw = mean_pooling(model_output, encoded_input['attention_mask'])
  sentence_embeddings = sentence_embeddings_raw.tolist()

  return sentence_embeddings[0]

In [51]:
dff['refined_tokens'][1]

['paris jackson cara',
 'delevingne enjoy night',
 'out in matching',
 'outfits they have',
 'amazing chemistry']

In [52]:
len(get_mean_pooling_emb(dff['refined_tokens'][0]))

768

In [53]:
# Generate Embeddings
dff['embedding']  = dff['refined_tokens'].progress_apply(get_mean_pooling_emb_tokenized)

  0%|          | 0/21718 [00:00<?, ?it/s]

In [54]:
dff['embedding_text']  = dff['title'].progress_apply(get_mean_pooling_emb)

  0%|          | 0/21718 [00:00<?, ?it/s]

In [55]:
len(dff['embedding'][0])

3

In [56]:
len(dff['embedding_text'][0])

768

In [57]:
dff.tail()

,title,label,id_1,source,id,tokens,index,refined_tokens,embedding,embedding_text
21713,the obameter: introduce a comprehensive immigr...,Real,23189,politifact,politifact6931,the obameter introduce a comprehensive immigra...,21713,"[the obameter introduce, a comprehensive immig...","[[-0.7559992074966431, 0.01481147576123476, 0....","[-0.5616924166679382, -0.35509511828422546, 0...."
21714,"trump asking congress, not mexico, to pay for ...",Real,23190,politifact,politifact13619,trump asking congress not mexico to pay for bo...,21714,"[trump asking congress, not mexico to, pay for...","[[-0.09557013213634491, -0.2762383222579956, -...","[-0.375062495470047, -0.2861070930957794, -0.1..."
21715,flake: “religious tests should have no place i...,Real,23191,politifact,politifact14731,flake religious tests should have no place in ...,21715,"[flake religious tests, should have no, place ...","[[0.23345446586608887, -0.3362610340118408, -0...","[0.1313381791114807, -0.04562729597091675, -0...."
21716,deputy director of national health statistics ...,Real,23193,politifact,politifact1576,deputy director of national health statistics ...,21716,"[deputy director of, national health statistic...","[[-0.6890026330947876, -0.06926846504211426, -...","[-0.7230011224746704, -0.230274960398674, -0.1..."
21717,romneys prolife conversion myth or reality jun...,Real,23194,politifact,politifact4720,romneys prolife conversion myth or reality june,21717,"[romneys prolife conversion, myth or reality, ...","[[-0.3077794909477234, -0.4073433578014374, 0....","[0.6357845067977905, -0.17522108554840088, 0.0..."


In [58]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21718 entries, 0 to 21717
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           21718 non-null  object
 1   label           21718 non-null  object
 2   id_1            21718 non-null  int64 
 3   source          21718 non-null  object
 4   id              21718 non-null  object
 5   tokens          21718 non-null  object
 6   index           21718 non-null  int64 
 7   refined_tokens  21718 non-null  object
 8   embedding       21718 non-null  object
 9   embedding_text  21718 non-null  object
dtypes: int64(2), object(8)
memory usage: 1.7+ MB


#Emotion Extraction Pipeline

In [59]:
pandarallel.initialize()

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [60]:
def get_nearest_neighbours_tokenized(index):
    # t1 = datetime.now()

    total_dict = []
    for i in range(len(dff['refined_tokens'][index])):
      tuples = []
      
      embeding = dff['embedding'][index][i]
      sentence_tokens = dff['refined_tokens'][index][i].split(' ')

      # for i,row_e in df.iterrows():
          
      #     dis = cosine_similarity([row_e['embedding']], [embeding])
      #     # print([row_e['tokens'],row_d['tokens'],dis])
      #     tuples.append([row_e['word'],row_e['emotion'],dis,row_e['embedding'],row_e['emotion-intensity-score']])

      indexes, distances = t.get_nns_by_vector(embeding, 50, include_distances=True)
      for i in range(len(indexes)):
        tuples.append([df['word'][indexes[i]],df['emotion'][indexes[i]],distances[i],df['embedding'][indexes[i]],df['emotion-intensity-score'][indexes[i]]
                      #  df['Positive'][indexes[i]],df['Negative'][indexes[i]]
                      ])
      # print(indexes)
      # print(distances)
      # print(tuples)
      
      s_tup = sorted(tuples, key=lambda x: x[2])#sort tuples based on the cosine distance
      neaarest_neighbs_words = []
      neaarest_neighbs_embs = []
      neaarest_neighbs_labels = []
      neaarest_neighbs_distance = []
      # neaarest_neighbs_positive = []
      # neaarest_neighbs_negative = []
      neaarest_neighbs_intensity_score = []
      for i,m in enumerate(s_tup[::-1]):
          # print(m)
          if(i<50):#getting the nearest 50 neighbours
              neaarest_neighbs_words.append(m[0])
              neaarest_neighbs_embs.append(m[3])
              neaarest_neighbs_labels.append(m[1])
              # distance = m[2].tolist()[0][0]
              distance = m[2]
              neaarest_neighbs_distance.append(distance)
              neaarest_neighbs_intensity_score.append(m[4])
              # neaarest_neighbs_positive.append(m[5])
              # neaarest_neighbs_negative.append(m[6])
    

      n_score_dict = calculate_scores(neaarest_neighbs_words,neaarest_neighbs_labels,neaarest_neighbs_distance,neaarest_neighbs_intensity_score
                                      )
      
      for key in n_score_dict:
        n_score_dict[key] = round((n_score_dict[key]/len(sentence_tokens)),4)
      
      # neighbour_output = [n_score_dict,{'words':neaarest_neighbs_words,'embs':neaarest_neighbs_embs,'labels':neaarest_neighbs_labels}]
      # normalized_score_dict = neighbour_output[0]
      
      n_score_dict = {k: round(v,3) for k, v in sorted(n_score_dict.items(), key=lambda item: item[1])}
      total_dict.append(n_score_dict)

    return total_dict

In [61]:
import operator

def dict_to_result_tokenized(list_emotion_dict):
  high_8 = []
  for i in range(len(list_emotion_dict)):
    emotion_dict = list_emotion_dict[i]
    if (all(value == 0 for value in emotion_dict.values())):
      highest_8 = None
    else:
      highest_8 = max(emotion_dict.items(), key=operator.itemgetter(1))[0]
    high_8.append(highest_8)
  return high_8

# result = emotion_candidates_recognition('That was good',1)
# x = dict_to_result(result)

In [62]:
def get_nearest_neighbours(index):
    # t1 = datetime.now()
    tuples = []
    
    embeding = dff['embedding_text'][index]
    sentence_tokens = dff['title'][index].split(' ')

    # for i,row_e in df.iterrows():
        
    #     dis = cosine_similarity([row_e['embedding']], [embeding])
    #     # print([row_e['tokens'],row_d['tokens'],dis])
    #     tuples.append([row_e['word'],row_e['emotion'],dis,row_e['embedding'],row_e['emotion-intensity-score']])

    indexes, distances = t.get_nns_by_vector(embeding, 50, include_distances=True)
    for i in range(len(indexes)):
      tuples.append([df['word'][indexes[i]],df['emotion'][indexes[i]],distances[i],df['embedding'][indexes[i]],df['emotion-intensity-score'][indexes[i]]
                    #  df['Positive'][indexes[i]],df['Negative'][indexes[i]]
                     ])
    # print(indexes)
    # print(distances)
    # print(tuples)
    
    s_tup = sorted(tuples, key=lambda x: x[2])#sort tuples based on the cosine distance
    neaarest_neighbs_words = []
    neaarest_neighbs_embs = []
    neaarest_neighbs_labels = []
    neaarest_neighbs_distance = []
    # neaarest_neighbs_positive = []
    # neaarest_neighbs_negative = []
    neaarest_neighbs_intensity_score = []
    for i,m in enumerate(s_tup[::-1]):
        # print(m)
        if(i<50):#getting the nearest 50 neighbours
            neaarest_neighbs_words.append(m[0])
            neaarest_neighbs_embs.append(m[3])
            neaarest_neighbs_labels.append(m[1])
            # distance = m[2].tolist()[0][0]
            distance = m[2]
            neaarest_neighbs_distance.append(distance)
            neaarest_neighbs_intensity_score.append(m[4])
            # neaarest_neighbs_positive.append(m[5])
            # neaarest_neighbs_negative.append(m[6])

    n_score_dict = calculate_scores(neaarest_neighbs_words,neaarest_neighbs_labels,neaarest_neighbs_distance,neaarest_neighbs_intensity_score
                                    # neaarest_neighbs_positive,neaarest_neighbs_negative
                                    )
    
    neighbour_output = [n_score_dict,{'words':neaarest_neighbs_words,'embs':neaarest_neighbs_embs,'labels':neaarest_neighbs_labels}]
    normalized_score_dict = neighbour_output[0]
  
    for key in normalized_score_dict:    
      normalized_score_dict[key] = round((normalized_score_dict[key]/len(sentence_tokens)),4)

    normalized_score_dict = {k: round(v,3) for k, v in sorted(normalized_score_dict.items(), key=lambda item: item[1])}
    # print(normalized_score_dict)
    return normalized_score_dict

In [63]:
def calculate_scores(neaarest_neighbs_words,neaarest_neighbs_labels,neaarest_neighbs_distance,neaarest_neighbs_intensity_score
                    #  ,neaarest_neighbs_positive,neaarest_neighbs_negative
                     ):
  score_dict = {
              'anticipation':0,
              'anger':0,
              'fear':0,
              'sadness':0,
              'trust':0,
              'joy':0,
              'surprise':0,
              'disgust':0
              # 'positive':0,
              # 'negative':0
              }

  #Scoring Mechanism
  for i in range(0,len(neaarest_neighbs_words)):
    distance = 0
    if neaarest_neighbs_distance[i] == 0 :
      distance = 1
    else :
      distance = (1/neaarest_neighbs_distance[i])
    
    score = (distance*(neaarest_neighbs_intensity_score[i]))  

    score_dict[neaarest_neighbs_labels[i]]=score_dict[neaarest_neighbs_labels[i]]+score

  # score_dict['positive']=score_dict['anger']+ score_dict['fear']+score_dict['disgust']+score_dict['sadness']
  # score_dict['negative']=score_dict['joy']+score_dict['trust']+score_dict['anticipation']+score_dict['surprise']




  #Normalising Mechanism
  normalized_score_dict = score_dict.copy()
  # for k in score_dict.keys():
  #   if score_dict[k] ==0:
  #     continue
  #     # del normalized_score_dict[k]
  #   else:
  #     normalized_score_dict[k]
  #     # normalized_score_dict[k] = round((score_dict[k]/score_max),3)

  return normalized_score_dict

In [64]:
import operator

def dict_to_result(emotion_dict):
  # del emotion_dict['positive']
  # del emotion_dict['negative']
  if (all(value == 0 for value in emotion_dict.values())):
    highest_8 = None
  else:
    highest_8 = max(emotion_dict.items(), key=operator.itemgetter(1))[0]
  return highest_8

# result = emotion_candidates_recognition('That was good',1)
# x = dict_to_result(result)

In [65]:
dff.tail()

,title,label,id_1,source,id,tokens,index,refined_tokens,embedding,embedding_text
21713,the obameter: introduce a comprehensive immigr...,Real,23189,politifact,politifact6931,the obameter introduce a comprehensive immigra...,21713,"[the obameter introduce, a comprehensive immig...","[[-0.7559992074966431, 0.01481147576123476, 0....","[-0.5616924166679382, -0.35509511828422546, 0...."
21714,"trump asking congress, not mexico, to pay for ...",Real,23190,politifact,politifact13619,trump asking congress not mexico to pay for bo...,21714,"[trump asking congress, not mexico to, pay for...","[[-0.09557013213634491, -0.2762383222579956, -...","[-0.375062495470047, -0.2861070930957794, -0.1..."
21715,flake: “religious tests should have no place i...,Real,23191,politifact,politifact14731,flake religious tests should have no place in ...,21715,"[flake religious tests, should have no, place ...","[[0.23345446586608887, -0.3362610340118408, -0...","[0.1313381791114807, -0.04562729597091675, -0...."
21716,deputy director of national health statistics ...,Real,23193,politifact,politifact1576,deputy director of national health statistics ...,21716,"[deputy director of, national health statistic...","[[-0.6890026330947876, -0.06926846504211426, -...","[-0.7230011224746704, -0.230274960398674, -0.1..."
21717,romneys prolife conversion myth or reality jun...,Real,23194,politifact,politifact4720,romneys prolife conversion myth or reality june,21717,"[romneys prolife conversion, myth or reality, ...","[[-0.3077794909477234, -0.4073433578014374, 0....","[0.6357845067977905, -0.17522108554840088, 0.0..."


In [66]:
dff['emotion_score_dict'] = dff['index'].parallel_apply(get_nearest_neighbours)

In [67]:
dff['emotion_score_dict_tokenized'] = dff['index'].parallel_apply(get_nearest_neighbours_tokenized)

In [68]:
print(dff['emotion_score_dict'][0])
print(dff['emotion_score_dict_tokenized'][0])

{'trust': 0.0, 'surprise': 0.129, 'disgust': 0.13, 'anger': 0.219, 'sadness': 0.225, 'fear': 0.602, 'anticipation': 0.832, 'joy': 0.959}
[{'trust': 0.0, 'disgust': 0.431, 'surprise': 0.478, 'sadness': 0.616, 'anger': 1.189, 'anticipation': 2.252, 'fear': 2.503, 'joy': 3.03}, {'trust': 0.0, 'surprise': 0.0, 'disgust': 0.678, 'sadness': 0.824, 'fear': 1.717, 'anger': 1.736, 'joy': 3.486, 'anticipation': 4.848}, {'trust': 0.0, 'disgust': 0.244, 'surprise': 0.324, 'anger': 0.445, 'sadness': 0.488, 'fear': 0.531, 'joy': 4.898, 'anticipation': 5.04}]


In [69]:
dff['highest_eight_label'] = dff['emotion_score_dict'].parallel_apply(dict_to_result)

In [70]:
def most_frequent(List):
    return max(set(List), key = List.count)
dff['sequence_emotion'] = dff['emotion_score_dict_tokenized'].parallel_apply(dict_to_result_tokenized)
dff['highest_eight_label_tokenized'] = dff['sequence_emotion'].apply(most_frequent)

In [71]:
from collections import Counter

def con_dicts(list_dicts):
  c = Counter()
  for d in list_dicts:
      c.update(d)
  final = dict(c)
  final = {k: round(v,3) for k, v in sorted(final.items(), key=lambda item: item[1])}
  return final

dff['cum_emotion_score_dict'] = dff['emotion_score_dict_tokenized'].apply(con_dicts)
print(dff['cum_emotion_score_dict'][0])

{'trust': 0.0, 'surprise': 0.802, 'disgust': 1.353, 'sadness': 1.928, 'anger': 3.37, 'fear': 4.751, 'joy': 11.414, 'anticipation': 12.14}


In [72]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21718 entries, 0 to 21717
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   title                          21718 non-null  object
 1   label                          21718 non-null  object
 2   id_1                           21718 non-null  int64 
 3   source                         21718 non-null  object
 4   id                             21718 non-null  object
 5   tokens                         21718 non-null  object
 6   index                          21718 non-null  int64 
 7   refined_tokens                 21718 non-null  object
 8   embedding                      21718 non-null  object
 9   embedding_text                 21718 non-null  object
 10  emotion_score_dict             21718 non-null  object
 11  emotion_score_dict_tokenized   21718 non-null  object
 12  highest_eight_label            21718 non-null  object
 13  s

In [73]:
dff.head()

,title,label,id_1,source,id,tokens,index,refined_tokens,embedding,embedding_text,emotion_score_dict,emotion_score_dict_tokenized,highest_eight_label,sequence_emotion,highest_eight_label_tokenized,cum_emotion_score_dict
0,did miley cyrus and liam hemsworth secretly ge...,Fake,0,gossipcop,gossipcop-2493749932,did miley cyrus and liam hemsworth secretly ge...,0,"[did miley cyrus, and liam hemsworth, secretly...","[[0.42651495337486267, -0.6673477292060852, 0....","[0.6491187214851379, -0.9568117260932922, 0.21...","{'trust': 0.0, 'surprise': 0.129, 'disgust': 0...","[{'trust': 0.0, 'disgust': 0.431, 'surprise': ...",joy,"[joy, anticipation, anticipation]",anticipation,"{'trust': 0.0, 'surprise': 0.802, 'disgust': 1..."
1,paris jackson & cara delevingne enjoy night ou...,Fake,1,gossipcop,gossipcop-4580247171,paris jackson cara delevingne enjoy night out ...,1,"[paris jackson cara, delevingne enjoy night, o...","[[-0.433077871799469, -0.4510994851589203, 0.5...","[-0.07626219093799591, -0.549978494644165, 1.2...","{'fear': 0.0, 'trust': 0.0, 'sadness': 0.013, ...","[{'sadness': 0.0, 'trust': 0.0, 'disgust': 0.5...",joy,"[joy, joy, anticipation, anticipation, joy]",joy,"{'trust': 0.0, 'sadness': 0.958, 'disgust': 2...."
2,celebrities join tax march in protest of donal...,Fake,2,gossipcop,gossipcop-941805037,celebrities join tax march in protest of donal...,2,"[celebrities join tax, march in protest, of do...","[[0.0017390132416039705, -0.6564664840698242, ...","[-0.043154940009117126, -0.4670027792453766, 0...","{'trust': 0.0, 'sadness': 0.021, 'anticipation...","[{'trust': 0.0, 'surprise': 0.0, 'anger': 0.11...",anger,"[anticipation, anger, fear]",anger,"{'trust': 0.0, 'sadness': 0.832, 'disgust': 1...."
3,cindy crawford's daughter kaia gerber wears a ...,Fake,3,gossipcop,gossipcop-2547891536,cindy crawford s daughter kaia gerber wears a ...,3,"[cindy crawford s, daughter kaia gerber, wears...","[[-0.3848035931587219, -0.39257967472076416, 0...","[-0.6619102358818054, -0.3519720137119293, 0.2...","{'trust': 0.0, 'surprise': 0.054, 'disgust': 0...","[{'trust': 0.0, 'sadness': 0.235, 'disgust': 0...",joy,"[joy, joy, anticipation, anticipation, joy]",joy,"{'trust': 0.0, 'sadness': 3.252, 'surprise': 3..."
4,full list of 2018 oscar nominations – variety,Fake,4,gossipcop,gossipcop-5476631226,full list of oscar nominations variety,4,"[full list of, oscar nominations variety]","[[-0.618384838104248, -0.6462655663490295, 0.1...","[-0.0010920644272118807, -0.22439801692962646,...","{'trust': 0.0, 'anger': 0.1, 'surprise': 0.222...","[{'trust': 0.0, 'anger': 0.481, 'sadness': 0.4...",joy,"[anticipation, joy]",joy,"{'trust': 0.0, 'anger': 0.687, 'surprise': 1.0..."


In [74]:
out_dff = dff.copy()
out_dff.drop(['emotion_score_dict_tokenized','index'], axis=1, inplace=True)

In [75]:
out_dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21718 entries, 0 to 21717
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   title                          21718 non-null  object
 1   label                          21718 non-null  object
 2   id_1                           21718 non-null  int64 
 3   source                         21718 non-null  object
 4   id                             21718 non-null  object
 5   tokens                         21718 non-null  object
 6   refined_tokens                 21718 non-null  object
 7   embedding                      21718 non-null  object
 8   embedding_text                 21718 non-null  object
 9   emotion_score_dict             21718 non-null  object
 10  highest_eight_label            21718 non-null  object
 11  sequence_emotion               21718 non-null  object
 12  highest_eight_label_tokenized  21718 non-null  object
 13  c

In [76]:
out_dff.to_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Results/Explainability/fakenewsnet.csv",index=False)